# Uppskatta storleken på skogsbränder
Innan du börjar den här uppgiften är det bra om du har gjort en tidigare uppgift som heter Torkan A.

I den här uppgiften kommer du få lära dig mer om polygoner och hur man kan använda dessa för att uppskatta storleken på en skogsbrand.  Till att börja med kommer du få bekanta dig lite med polygoner. Börja med att köra koden nedan.





In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Wedge, Polygon
from matplotlib.collections import PatchCollection

points = [[2, 1], [8, 1], [8, 4]]
polygon = plt.Polygon(points)
plt.xticks(np.arange(0, 11, 1))
plt.yticks(np.arange(0, 11, 1))
plt.grid()
plt.gca().add_patch(polygon)
plt.show()

En triangel med hörn i `[2, 1], [8, 1], [8, 4]` har bildats. Polygoner är geometriska former med hörn.
Nu ska du få prova på att skapa och plotta din egen polygon. Skriv in egna koordinater där du tycker polygonens hörn ska vara. 
En koordinat ska skrivas in på formen `[x,y]`.
Du kan enkelt utöka antalet hörn hos polygonen genom att utöka storleken på listan. När du är klar behöver du även ändra intervallen på x- och y-axlarna så att de passar med dina koordinater. Har du exempelvis ett hörn placerat i `[-91,205]` och ett annat i `[-9,3]` kan du ha intervallet `[-100,0]` på x-axeln och `[0,210]` på y-axeln. Det viktigaste är att alla koordinater du har valt innefattas av intervallet så att de syns i figuren.

 
Du kan också ändra upplösningen på axlarna genom att ändra `interval_step` till ett tal som passar till dina intervall. Prova dig fram till något passande! Vad händer om ditt intervallsteg är större än maxvärdet på intervallet?

In [ ]:
#Skriv in koordinaterna på polygonens hörn:
corners = [[,],[,],[,],[,],  .....]

#Skriv även in dina intervall på x- och y-axlarna och deras intervallsteg:
min_x=
max_x=
x_interval_step=

min_y=
max_y=
y_interval_step=

polygon = plt.Polygon(corners)
plt.gca().add_patch(polygon)
plt.xticks(np.arange(min_x, max_x, x_interval_step))
plt.yticks(np.arange(min_y, max_y, y_interval_step))
plt.grid()
plt.show()

Nu när du fått bekanta dig lite grann med polygoner är nästa steg att undersöka och uppskatta storleken på en skogsbrand. Här finns satellitbilder samlade över två olika skogsbränder.

Välj vilken av dem du vill jobba med eller gör båda två om du har tid. Klicka på pilen till vänster om ett avsnitt för att öppna eller stänga det.

### Sverige- Skogsbranden i Kårböle 2018
2018 var ett år med många kraftiga skogsbränder i norra Sverige. Kårböle i Ljusdals kommun var särskilt drabbat. Kör nedanstående kodblock för att ladda ned två satellitbilder. Den ena är en bild över området hösten 2016, och den andra hösten 2019.

In [ ]:
!wget https://github.com/lunduniversity/schoolprog-satellite-data/raw/master/forest_fires/Sweden/bands_2019_swe.npz --quiet
!wget https://github.com/lunduniversity/schoolprog-satellite-data/raw/master/forest_fires/Sweden/bands_2016_swe.npz --quiet
!wget https://github.com/lunduniversity/schoolprog-satellite/raw/master/exercises/forest_fires/SWE_firepolygons.png --quiet


#Banden för den svenska skogsbranden
bands16 = np.load('bands_2016_swe.npz')
bands19 = np.load('bands_2019_swe.npz')

#Plockar ut banden var för sig som float
#Färgbanden för hösten 2016
blue16 = bands16["blue"].astype(float)
green16 = bands16["green"].astype(float)
red16 = bands16["red"].astype(float)
nir16 = bands16["nir"].astype(float)

#Färgbanden för hösten 2019
blue19 = bands19["blue"].astype(float)
green19 = bands19["green"].astype(float)
red19 = bands19["red"].astype(float)
nir19 = bands19["nir"].astype(float)

Till att börja med kan du undersöka färgbilder av området. För att skapa färgbilderna behöver du kombinera tre färgband som satelliten mäter. En vanlig färgbild är en kombination av rött, grönt och blått ljus.  Du kanske har lagt märke till att det finns ytterligare ett färgband som kallas nir. Nir är förkortning för Near InfraRed, vilket är det ljus som har lite längre våglängd än rött. Det går inte att se med blotta ögat men satelliten har instrument som kan plocka upp det. Nir kommer att behövas lite senare i uppgiften.

In [44]:
#Skapa en färgbild av det röda, gröna och blåa bandet

def color_img(red,green,blue):
  rgb=np.zeros([500,500,3],dtype=np.uint8)
  divide=8
  for x in range(500):
    for y in range(500):
      r = min(255, red[x][y]/divide)
      g = min(255, green[x][y]/divide)
      b = min(255, blue[x][y]/divide)
      rgb[x,y] = [r,g,b]
  return rgb




Funktionen `color_img(red,green,blue)` tar färgbanden som argument för att skapa bilden. Välj vilket år du vill titta på genom att ändra funktionens inmatning av färgband till motsvarande år. Kör sedan kodblocket nedan.

Undersök de två bilderna. Kan du se någon skillnad? 

In [ ]:
from PIL import Image

#Välj de färgband som motsvarar det årtal du vill kolla på. 
#Färgbanden ska vara i ordningen rött,grönt,blått.
rgb=color_img(xxx,xxx,xxx)


img = Image.fromarray(rgb)
display(img)



<details>
<summary markdown="span">
Svar
</summary>
<p>

För att se färgbilden för hösten 2016:
<pre><code>rgb=color_img(red16,green16,blue16)
</code></pre>
För att se färgbilden för hösten 2019:
<pre><code>rgb=color_img(red19,green19,blue19)
</code></pre>
</p>
</details>
<p></p>

I nästa steg ska vi skapa en bild som presenterar Normalized Difference Vegetation Index (NDVI). Det är något som du borde ha stött på i tidigare uppgifter. Om du inte vet vad NDVI är så är det en god idé att repetera uppgiften Torkan A. 

NDVI räknas ut med hjälp av NIR och det röda bandet enligt följande formel: 

           NIR - Red
    NDVI = ---------
           NIR + Red

Skriv nu en funktion med namn `ndvi_img` som räknar ut NDVI i följande kodblock. Om du är osäker på syntaxen får du ett färdigt kodskelett under tipsfliken, men du behöver själv skriva in vilka argument funktionen behöver och hur NDVI ska räknas ut. Om du har gjort funktioner innan så prova gärna själv innan du kikar på tipset. Var noga med placeringen av paranteser!

In [ ]:
#Skriv din funktion ndvi_img här:
def ...



<details>
<summary markdown="span">
Tips!
</summary>
<p><pre><code>def ndvi_img(xxx,xxx):
  #skriv hur ndvi räknas ut här:
  ndvi=
  return ndvi</code></pre>
</p>
</details>
<p></p>

<details>
<summary markdown="span">
Lösning
</summary>
<p><pre><code>def ndvi_img(nir,red):
 ndvi=(nir-red)/(nir+red)
 return ndvi</code></pre>
</p>
</details>
<p></p>

Nu kan du prova din funktion genom att se hur NDVI för området ser ut. Börja med år 2016 och fortsätt sedan med 2019. Skriv in rätt argument i din ndvi_img funktion och kör sedan kodblocket. Om du är osäker på namnen på färgbanden så kan du hitta dem i kodblocket som skapar färgbilderna.

In [ ]:
#Byt ut xxx mot namnen på rätt färgband:
ndvi16 = ndvi_img(xxx,xxx)

plt.figure(figsize=(10,10))
plt.pcolormesh(ndvi16, cmap='brg')
plt.rcParams['xtick.top'] = plt.rcParams['xtick.labeltop'] = True
plt.ylim(ndvi16.shape[0], 0)
plt.clim(-1.0, 0.7)
plt.colorbar(label='NDVI')
plt.show()

In [ ]:
#Byt ut xxx mot namnen på rätt färgband:
ndvi19 = ndvi_img(xxx,xxx)

plt.figure(figsize=(10,10))
plt.pcolormesh(ndvi19, cmap='brg')
plt.ylim(ndvi19.shape[0], 0)
plt.clim(-1.0, 0.7)
plt.colorbar(label='NDVI')
plt.rcParams['xtick.top'] = plt.rcParams['xtick.labeltop'] = True
plt.show()


<details>
<summary markdown="span">
Svar
</summary>
<p><pre><code> ndvi16=ndvi_img(nir16,red16)
 ndvi19=ndvi_img(nir19,red19)
</code></pre>
</p>
</details>
<p></p>


Som du kanske kommer ihåg från Torkan A så är NDVI ett sätt att dela upp en landyta i olika kategorier. Ett index som är över 0.1 kan i de flesta fall sägas vara någon form av vegetation. Ju högre NDVI desto tätare växtlighet. Ett NDVI-värde mellan -0.1 till 0.1 brukar indikera en landyta fri från vegetation. Negativa NDVI-värden mellan -0.1 till -1 brukar istället betyda att det är en yta täckt av vatten.

Jämför de båda bilderna. Är det lättare att se skillnader om man kollar på NDVI jämfört med färgbilder?
I vilka områden tror du skogsbranden har ägt rum?
Scrolla upp och kolla på färgbilderna igen, lokalisera var det finns moln. Kan du se molnen även i bilderna med NDVI? 


Som du kanske lagt märke till i NDVI-bilden från 2019 så är det främst två områden som är intressanta. Ett precis norr om älven och ett söder om. Vi behöver nu skapa två polygoner för att uppskatta storleken på dessa områden. 

Börja med att köra kodblocket nedan för att få upp en större bild av NDVI över området år 2019.



In [ ]:
plt.figure(figsize=(15,15))
plt.pcolormesh(ndvi19, cmap='brg')
plt.ylim(ndvi19.shape[0], 0)
plt.clim(-1.0, 0.7)
plt.colorbar(label='NDVI')
plt.xticks(np.arange(0, 500+1, 25.0))
plt.yticks(np.arange(0, 500+1, 25.0))
plt.grid()
plt.rcParams['xtick.top'] = plt.rcParams['xtick.labeltop'] = True 

#Bygg ut listan med fler kanter så att hela norra delen av branden täcks:
corner_north = [[140,45],[175,55],[200,54]]
polygon = plt.Polygon(corner_north,fill=None)
plt.gca().add_patch(polygon)

#Bygg ut listan med fler kanter så att hela södra delen av branden täcks:
corner_south=[[200,260],[275,320],[300,310]]
polygon = plt.Polygon(corner_south,fill=None)
plt.gca().add_patch(polygon)
plt.show()


Bilden som skapas ska användas för att bilda polygoner för att ringa in de två brandområdena. Bygg ut listorna `corner_north` och `corner_south` med koordinater så att hela området täcks. Du väljer själv hur noggrann du vill vara. Kortare avstånd mellan kanterna gör att du kan fånga små utbuktningar lättare. Runt 15 kanter och uppåt är ett bra riktmärke. Vi har börjat med tre kanter i vardera listan för att hjälpa dig lite på traven. Utnyttja koordinataxlarna på sidorna för att lokalisera var du vill placera ut nästa kant. Lägg märke till i vilket hörn origo är placerat innan du börjar. Det är viktigt att du skriver in kanterna i rätt ordning så att linjerna mellan kanterna dras rätt. Vi har valt att röra oss medsols runt brandområdet, så det är viktigt att du fortsätter med detta om du vill utnyttja de färdiglagda koordinaterna. Lokalisera den sista punkten i listan och utgå från den. Kör kodstycket med jämna mellanrum för att se hur polygonen utvecklas och om du behöver korrigera kanterna i nån riktning.
Kör kodblocket nedan för att se en referensbild. Den visar hur din egen bild borde se ut när du är klar.

In [ ]:
import matplotlib.image as img 
# reading png image file 
fire = img.imread('SWE_firepolygons.png') 
plt.figure(figsize = (15,15))
plt.imshow(fire)
plt.axis('off')
plt.show()

Funktionen `area_calc` kan användas för att räkna ut pixelarean till dina två polygoner. Parameter till funktionen är en lista med koordinater för dina polygonkanter. Skriv in rätt parametervärde till `area_calc` för att räkna ut arean i kodblocket nedan. Räkna sedan ut totalarean.

 

In [ ]:
def area_calc(corners):
  area=0.0
  n=len(corners)
  for i in range(n):
    i1=(i+1)%n
    area += corners[i][0]*corners[i1][1] - corners[i1][0]*corners[i][1]
  area *=0.5
  return abs(area)

#Byt ut xxx mot rätt parametrar
area_north=area_calc(xxx)
area_south=area_calc(xxx)



totalarea=area_north+area_south
print(totalarea, 'pixlar')


<details>
<summary markdown="span">
Svar
</summary>
<p><pre><code>area_north=area_calc(corner_north)
area_south=area_calc(corner_south)</code></pre>
Arean borde vara någonstans mellan 38 000 - 58 000.
</p>
</details>
<p></p>

I nästa steg vill du omvandla pixlarna du räknat fram till $km^2$. En pixel i bilden motsvarar ett område på 40x40 $m$ vilket i sin tur motsvarar 1600 $m^2$. Om du är osäker på omvandlingen kan du börja med att räkna ut hur många $m^2$ brandområdet motsvarar. Gör detta i kodblocket nedan.

In [ ]:
area_m2= 
print(area_m2,'m^2')

<details>
<summary markdown="span">
Lösning
</summary>
<p><pre><code> area_m2=totalarea*1600
</code></pre>
</p>
</details>
<p></p>

Sen behöver du fundera ut hur arean kan omvandlas till $km^2$. Skriv in din omvandling i kodblocket nedan. 

In [ ]:
area_km2 = 
print(area_km2, 'km^2')

<details>
<summary markdown="span">
Tips!
</summary>
<p>Som du säkert redan vet är 1000 meter samma som 1 km. 1 km^2 är då samma som 1000^2 m^2, alltså 1000000 m^2.
</p>
</details>
<p></p>

<details>
<summary markdown="span">
Lösning
</summary>
<p><pre><code> area_km2=area_m2/1000000
</code></pre>
</p>
</details>
<p></p>

### Australien- Skogsbranden på Kangaroo Island 2019
Årsskiftet 2019-2020 skedde en brandkatastrof i stora delar av Australien. I den här uppgiften ska du få undersöka brandområdet på Kangaroo Island, en liten ö som ligger längs Australiens södra kust. Kör nedanstående kodblock för att ladda ned två satellitbilder. Den ena är en bild över området december 2019, och den andra januari 2020.

In [ ]:

!wget https://github.com/lunduniversity/schoolprog-satellite-data/raw/master/forest_fires/Australia/bands_2019_aus.npz --quiet
!wget https://github.com/lunduniversity/schoolprog-satellite-data/raw/master/forest_fires/Australia/bands_2020_aus.npz --quiet
!wget https://github.com/lunduniversity/schoolprog-satellite/raw/master/exercises/forest_fires/AUS_firepolygons.png --quiet

#Banden för den australiska skogsbranden
bands19 = np.load('bands_2019_aus.npz')
bands20 = np.load('bands_2020_aus.npz')


#Skapa en färgbild av det röda, gröna och blåa bandet

def color_img(red,green,blue):
  rgb=np.zeros([500,625,3],dtype=np.uint8)
  divide=8
  for x in range(500):
    for y in range(625):
      r = min(255, red[x][y]/divide)
      g = min(255, green[x][y]/divide)
      b = min(255, blue[x][y]/divide)
      rgb[x,y] = [r,g,b]
  return rgb


Till att börja med kan du undersöka färgbilder av området. För att skapa färgbilderna behöver du kombinera tre färgband som satelliten mäter. En vanlig färgbild är en kombination av rött, grönt och blått ljus.  Du kanske har lagt märke till att det finns ytterligare ett färgband som kallas nir. Nir är förkortning för Near InfraRed, vilket är det ljus som har lite längre våglängd än rött. Det går inte att se med blotta ögat men satelliten har instrument som kan plocka upp det. Nir kommer att behövas lite senare i uppgiften.

Funktionen `color_img(red,green,blue)` tar färgbanden som argument för att skapa bilden. Välj vilket år du vill titta på genom att ändra funktionens inmatning av färgband till motsvarande år. Kör sedan kodblocket nedan.

Undersök de två bilderna. Kan du se någon skillnad? 

In [ ]:
from PIL import Image

#Plockar ut banden var för sig som float
#Färgbanden för december 2019
blue19=bands19["blue"].astype(float)
green19=bands19["green"].astype(float)
red19 = bands19["red"].astype(float)
nir19 = bands19["nir"].astype(float)

#Färgbanden för januari 2020
blue20=bands20["blue"].astype(float)
green20=bands20["green"].astype(float)
red20 = bands20["red"].astype(float)
nir20 = bands20["nir"].astype(float)

#Välj de färgband som motsvarar det årtal du vill kolla på. 
#Färgbanden ska vara i ordningen rött,grönt,blått.
rgb=color_img(xxx,xxx,xxx)


img = Image.fromarray(rgb)
display(img)

<details>
<summary markdown="span">
Svar
</summary>
<p>

För att se färgbilden för december 2019:
<pre><code>rgb=color_img(red19,green19,blue19)</code></pre>
För att se färgbilden för januari 2020:
<pre><code>rgb=color_img(red20,green20,blue20)</code></pre>
</p>
</details>
<p></p>

I nästa steg ska vi skapa en bild som presenterar Normalized Difference Vegetation Index (NDVI). Det är något som du borde ha stött på i tidigare uppgifter. Om du inte vet vad NDVI är så är det en god idé att repetera uppgiften Torkan A. 

NDVI räknas ut med hjälp av NIR och det röda bandet enligt följande formel: 

           NIR - Red
    NDVI = ---------
           NIR + Red

Skriv nu en funktion med namn `ndvi_img` som räknar ut NDVI i följande kodblock. Om du är osäker på syntaxen får du ett färdigt kodskelett under tipsfliken, men du behöver själv skriva in vilka argument funktionen behöver och hur NDVI ska räknas ut. Om du har gjort funktioner innan så prova gärna själv innan du kikar på tipset. Var noga med placeringen av paranteser!

In [ ]:
#Skriv din funktion ndvi_img här:
def ...

<details>
<summary markdown="span">
Tips!
</summary>
<p><pre><code>def ndvi_img(xxx,xxx):
  #skriv hur ndvi räknas ut här:
  ndvi=
  return ndvi
</code></pre>
</p>
</details>
<p></p>

<details>
<summary markdown="span">
Lösning
</summary>
<p><pre><code>def ndvi_img(nir,red):
 ndvi=(nir-red)/(nir+red)
 return ndvi
</code></pre>
</p>
</details>
<p></p>

Nu kan du prova din funktion genom att se hur NDVI för området ser ut. Börja med år 2019 och fortsätt sedan med 2020. Skriv in rätt argument i din ndvi_img funktion och kör sedan kodblocket. Om du är osäker på namnen på färgbanden så kan du hitta dem i kodblocket som skapar färgbilderna.

In [ ]:
#Byt ut xxx mot namnen på rätt färgband:
ndvi19 = ndvi_img(xxx,xxx)

plt.figure(figsize=(10,10))
plt.pcolormesh(ndvi19, cmap='brg')
plt.rcParams['xtick.top'] = plt.rcParams['xtick.labeltop'] = True
plt.ylim(ndvi19.shape[0], 0)
plt.clim(-1.0, 0.7)
plt.colorbar(label='NDVI')
plt.show()

In [ ]:
#Byt ut xxx mot namnen på rätt färgband:
ndvi20 = ndvi_img(xxx,xxx)

plt.figure(figsize=(10,10))
plt.pcolormesh(ndvi20, cmap='brg')
plt.rcParams['xtick.top'] = plt.rcParams['xtick.labeltop'] = True
plt.ylim(ndvi20.shape[0], 0)
plt.clim(-1.0, 0.7)
plt.colorbar(label='NDVI')
plt.show()

<details>
<summary markdown="span">
Svar
</summary>
<p><pre><code> ndvi19=ndvi_img(nir19,red19)
 ndvi20=ndvi_img(nir20,red20)
</code></pre>
</p>
</details>
<p></p>


Som du kanske kommer ihåg från Torkan A så är NDVI ett sätt att dela upp en landyta i olika kategorier. Ett index som är över 0.1 kan i de flesta fall sägas vara någon form av vegetation. Ju högre NDVI desto tätare växtlighet. Ett NDVI-värde mellan -0.1 till 0.1 brukar indikera en landyta fri från vegetation. Negativa NDVI-värden mellan -0.1 till -1 brukar istället betyda att det är en yta täckt av vatten.

Jämför de båda bilderna. Är det lättare att se skillnader om man kollar på NDVI jämfört med färgbilder?
I vilka områden tror du skogsbranden har ägt rum? Vad tror du orsakar de röda prickarna i vattnet?
Scrolla upp och jämför med färgbilderna, går det att se något där?

Börja med att köra kodblocket nedan för att få upp en större bild av NDVI över området år 2020.


In [ ]:
plt.figure(figsize=(15,15))
plt.pcolormesh(ndvi20, cmap='brg')
plt.ylim(ndvi20.shape[0], 0)
plt.clim(-1.0, 0.7)
plt.colorbar(label='NDVI')
plt.xticks(np.arange(0, 625+1, 25.0))
plt.yticks(np.arange(0, 500+1, 25.0))
plt.grid()
plt.rcParams['xtick.top'] = plt.rcParams['xtick.labeltop'] = True 

#Bygg ut listan med fler kanter så att hela brandområdet täcks:
corners = [[540, 130],[525, 155],[510, 190],[540, 200],[570, 210],
           [590, 120],[620, 115],[620, 250],[580, 280],[550, 280],
           [510, 260],[500, 230],[475, 225],[460, 260],[420, 300],
           [460, 310],[460, 330],[425, 325],[450, 350],[475, 355]]

             
           
polygon = plt.Polygon(corners,fill=None)
plt.gca().add_patch(polygon)
plt.show()

Bilden som skapas ska användas för att bilda en polygon för att ringa in brandområdet. Bygg ut listan `corners`  med koordinater så att hela området täcks. Du väljer själv hur noggrann du vill vara. Kortare avstånd mellan kanterna gör att du kan fånga små utbuktningar lättare. Vi har börjat med 20 kanter i listan för att hjälpa dig lite på traven, minst 30 kanter till behövs läggas in för att hela området ska täckas. Utnyttja koordinataxlarna på sidorna för att lokalisera var du vill placera ut nästa kant. Lägg märke till i vilket hörn origo är placerat innan du börjar. Det är viktigt att du skriver in kanterna i rätt ordning så att linjerna mellan kanterna dras rätt. Vi har valt att röra oss medsols runt brandområdet, så det är viktigt att du fortsätter med detta om du vill utnyttja de färdiglagda koordinaterna. Lokalisera den sista punkten i listan och utgå från den. Kör kodstycket med jämna mellanrum för att se hur polygonen utvecklas och om du behöver korrigera kanterna i nån riktning.
Kör kodblocket nedan för att se en referensbild. Den visar hur din egen bild borde se ut när du är klar.

In [ ]:
import matplotlib.image as img 
# reading png image file 
fire = img.imread('AUS_firepolygons.png') 
plt.figure(figsize = (15,15))
plt.imshow(fire)
plt.axis('off')
plt.show()

Funktionen `area_calc` kan användas för att räkna ut pixelarean till dina två polygoner. Parameter till funktionen är en lista med koordinater för dina polygonkanter. Skriv in rätt parametervärde till `area_calc` för att räkna ut arean i kodblocket nedan. Räkna sedan ut totalarean.

In [ ]:
def area_calc(corners):
  area=0.0
  n=len(corners)
  for i in range(n):
    i1=(i+1)%n
    area += corners[i][0]*corners[i1][1] - corners[i1][0]*corners[i][1]
  area *=0.5
  return abs(area)
 

#Byt ut xxx mot rätt parametrar
totalarea =area_calc(xxx)

print(totalarea, 'pixlar')

<details>
<summary markdown="span">
Svar
</summary>
<p><pre><code>area=area_calc(corners)</code></pre>
Arean borde vara någonstans mellan 130 000 - 150 000 pixlar.
</p>
</details>
<p></p>

I nästa steg vill du omvandla pixlarna du räknat fram till $km^2$. En pixel i bilden motsvarar 14400 $m^2$. Om du är osäker på omvandlingen kan du börja med att räkna ut hur många $m^2$ brandområdet motsvarar. Gör detta i kodblocket nedan.

In [ ]:
area_m2= 
print(area_m2,'m^2')

<details>
<summary markdown="span">
Lösning
</summary>
<p><pre><code> area_m2=totalarea*14400
</code></pre>
</p>
</details>
<p></p>

Sen behöver du fundera ut hur arean kan omvandlas till $km^2$. Skriv in din omvandling i kodblocket nedan. 

In [ ]:
area_km2 =
print(area_km2, 'km^2')

<details>
<summary markdown="span">
Tips!
</summary>
<p>Som du säkert redan vet är 1000 meter samma som 1 km. 1 km^2 är då samma som 1000^2 m^2, alltså 1000000 m^2.
</p>
</details>
<p></p>

<details>
<summary markdown="span">
Lösning
</summary>
<p><pre><code> area_km2=area_m2/1000000
</code></pre>
</p>
</details>
<p></p>

# Quiz

Kör kodrutan nedan för att starta quizzet. Kommer du ihåg vad vi har gått igenom?

In [ ]:
#@title Starta quizzet!
!wget https://raw.githubusercontent.com/lunduniversity/schoolprog-satellite/master/for_developers/quiz.py --quiet
!wget https://raw.githubusercontent.com/lunduniversity/schoolprog-satellite/master/exercises/forest_fires/quiz_skogsbrand.json --quiet
import json
import quiz

with open("quiz_skogsbrand.json") as f:
  quiz_dict = json.load(f)
quiz.main(quiz_dict)